In [35]:
# Starter cell: load logs, apply cleaning steps, and report stats
# Adjust DATA_DIR if your logs live somewhere else.

from pathlib import Path
import pandas as pd
import numpy as np
import re

# ──────────────────────────────────────────────────────────────────────────────
# Load all CSV logs (read as strings first so we can do exact/substring checks)
# ──────────────────────────────────────────────────────────────────────────────
DATA_DIR = Path("../data-2025")
if not DATA_DIR.exists():
    raise FileNotFoundError(f"Data directory not found: {DATA_DIR.resolve()}")

csv_paths = sorted(DATA_DIR.rglob("*.csv"))

frames = []
expected_cols = ["timestamp","mode","value","vbat_mV","vin_mV","iout_mA","soc_C","rp1_C","pmic_C","error"]

for p in csv_paths:
    try:
        tmp = pd.read_csv(p, dtype=str)   # keep everything as str for now
    except Exception as e:
        print(f"Warning: failed to read {p}: {e}")
        continue

    # Ensure expected columns exist
    for col in expected_cols:
        if col not in tmp.columns:
            tmp[col] = np.nan

    # Preserve raw value string EXACTLY as read (no strip/normalize yet)
    tmp["value_str"] = tmp["value"].astype(str)

    # Normalize text fields we care about
    tmp["mode"]  = tmp["mode"].astype(str).str.strip().str.lower()
    tmp["error"] = tmp["error"].astype(str).fillna("").str.strip()

    # Parse timestamp
    tmp["timestamp"] = pd.to_datetime(tmp["timestamp"], errors="coerce")

    # Keep source filename
    tmp["source_file"] = str(p)

    frames.append(tmp)

if frames:
    df = pd.concat(frames, ignore_index=True)
    df = df.dropna(subset=["timestamp"]).sort_values("timestamp")
else:
    df = pd.DataFrame(columns=expected_cols + ["value_str","source_file"])

print(f"Loaded {len(csv_paths)} CSV files; total rows: {len(df)}")

# ──────────────────────────────────────────────────────────────────────────────
# STEP 1: Apply cutoff date; report rows remaining
# ──────────────────────────────────────────────────────────────────────────────
CUTOFF_DATE = pd.Timestamp("2025-08-21")  # inclusive: keep rows >= 2025-08-21 00:00
before_rows = len(df)
df = df[df["timestamp"] >= CUTOFF_DATE].copy()
print(f"[Step 1] Applied cutoff at {CUTOFF_DATE.date()}: kept {len(df)} / {before_rows} rows")

# ──────────────────────────────────────────────────────────────────────────────
# STEP 2: Keep only rows where mode is exactly "watt"; report discarded & remaining
# ──────────────────────────────────────────────────────────────────────────────
pre_rows = len(df)
mode_mask = df["mode"].eq("watt")   # exact string match
discarded_non_watt = int((~mode_mask).sum())
df = df[mode_mask].copy()
print(f"[Step 2] Discarded non-'watt' modes: {discarded_non_watt}; rows remaining: {len(df)}")

# ──────────────────────────────────────────────────────────────────────────────
# STEP 3: Discard rows whose raw 'value' STRING contains "8888.8000"
#         (substring match; no numeric conversions; resilient to quotes/padding)
# ──────────────────────────────────────────────────────────────────────────────
pre_rows = len(df)
sentinel_mask = df["value_str"].astype(str).str.contains("8888.8000", regex=False, na=False)
discarded_sentinels = int(sentinel_mask.sum())
df = df[~sentinel_mask].copy()
print(f"[Step 3] Discarded rows where value contains '8888.8000': {discarded_sentinels}; rows remaining: {len(df)}")

# ──────────────────────────────────────────────────────────────────────────────
# STEP 4: Discard rows with decode/segment warnings *AND* zero/NaN reading
#         (e.g., "Warning: decode_digit got unrecognized segment pattern: ...")
#         This targets bogus zeros without dropping all rows that happen to have warnings.
# ──────────────────────────────────────────────────────────────────────────────
pre_rows = len(df)

# Build a temporary numeric value for zero/NaN checks (strip commas/whitespace safely)
value_norm_for_num = df["value_str"].str.replace(r"[^\d.\-]", "", regex=True)
value_num_tmp = pd.to_numeric(value_norm_for_num, errors="coerce")
zero_or_nan = value_num_tmp.isna() | value_num_tmp.eq(0)

# Error pattern indicating decoding/segment issues
err_decode_mask = df["error"].str.contains(r"(decode|segment|unrecognized|fail|parse)", case=False, na=False)

drop_mask = err_decode_mask & zero_or_nan
discarded_err_zero = int(drop_mask.sum())
df = df[~drop_mask].copy()
print(f"[Step 4] Discarded rows with decode/segment warnings AND zero/NaN value: {discarded_err_zero}; rows remaining: {len(df)}")

# ──────────────────────────────────────────────────────────────────────────────
# Additional summary (post-cleaning)
# ──────────────────────────────────────────────────────────────────────────────
# Convert numerics now that string-based filtering is done
for num_col in ["value","vbat_mV","vin_mV","iout_mA","soc_C","rp1_C","pmic_C"]:
    df[num_col] = pd.to_numeric(df[num_col], errors="coerce")

# Decode-error count & percentage (post-cleaning; informational only)
pat = re.compile(r"(?:decode.*digit|segment)", re.IGNORECASE)
decode_err_mask = df["error"].str.contains(pat, na=False)
decode_err_rows = int(decode_err_mask.sum())
total_rows = len(df)
decode_pct = (decode_err_rows / total_rows * 100.0) if total_rows else 0.0

# Single total runtime as D/H/M
if total_rows and df["timestamp"].notna().any():
    tmin = df["timestamp"].min()
    tmax = df["timestamp"].max()
    span = tmax - tmin
    total_seconds = int(span.total_seconds())
    days = total_seconds // 86400
    rem = total_seconds % 86400
    hours = rem // 3600
    rem %= 3600
    minutes = rem // 60
else:
    tmin = tmax = None
    days = hours = minutes = 0

print(f"\nCSV files represented (post-cleaning): {df['source_file'].nunique()}")
print(f"Segment-decode error rows (remaining): {decode_err_rows} ({decode_pct:.2f}% of total)")
print(f"Time span: {tmin} → {tmax}")
print(f"Total runtime: {days} days, {hours} hours, {minutes} minutes")

# `df` is now the cleaned DataFrame for plots/analysis.
# Example: power_df = df[['timestamp','value','soc_C']].rename(columns={'value':'watts'}).copy()

# OPTIONAL (uncomment if you truly never expect 0 W in valid data):
# zeros_left = int((df['value'] == 0).sum())
# if zeros_left:
#     print(f"Note: {zeros_left} zero-watt rows remain (no decode warning). Uncomment filter below to drop them.")
#     # df = df[df['value'] != 0].copy()


Loaded 32 CSV files; total rows: 234375
[Step 1] Applied cutoff at 2025-08-21: kept 230935 / 234375 rows
[Step 2] Discarded non-'watt' modes: 3869; rows remaining: 227066
[Step 3] Discarded rows where value contains '8888.8000': 0; rows remaining: 227066


C:\Users\Paul\AppData\Local\Temp\ipykernel_49496\513810914.py:96: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  err_decode_mask = df["error"].str.contains(r"(decode|segment|unrecognized|fail|parse)", case=False, na=False)


[Step 4] Discarded rows with decode/segment warnings AND zero/NaN value: 11760; rows remaining: 215306

CSV files represented (post-cleaning): 23
Segment-decode error rows (remaining): 0 (0.00% of total)
Time span: 2025-08-22 16:37:50.372000 → 2025-08-29 15:37:08.748000
Total runtime: 6 days, 22 hours, 59 minutes


In [38]:
# Chart: Watts vs Time (cleaned df) with option to plot ALL data or a SUBSET
# - When SUBSET is chosen, prints the relevant rows from df and saves them to CSV
# - 2000 x 1000 PNG, daily major ticks ("Friday 8/29"), faint hourly grid, night shading

import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter

# ───────────── CONFIG ─────────────
PLOT_SCOPE = "all" #"subset"   # "all" or "subset"

# If PLOT_SCOPE == "subset", choose ONE of the following selection styles:

# (A) Time window:
SUBSET_START = "2025-08-25 20:30:00"
SUBSET_END   = "2025-08-25 20:45:00"

# (B) Time around a center point (± minutes):
CENTER_TIME  = "2025-08-25 19:05:00"
RADIUS_MIN   = 10

# Selection style to use when PLOT_SCOPE == "subset": "window" or "around"
SUBSET_STYLE = "window"  # "window" or "around"

# Output / printing controls
OUT_PNG = "power_watts_subset_nightshade.png" if PLOT_SCOPE == "subset" else "power_watts_all_nightshade.png"
PRINT_MAX_ROWS = 500      # cap printed rows to avoid flooding the console
SAVE_SUBSET_CSV = True
SUBSET_CSV_PATH = "subset_rows_for_manual_check.csv"

# Night shading times (fixed for the whole range)
SUNRISE_STR = "06:22:28"
SUNSET_STR  = "19:31:14"

# Figure size for 2000 x 1000
FIGSIZE_IN = (10, 5)
DPI = 200
# ──────────────────────────────────

def _fmt_day(x, pos):
    d = mdates.num2date(x)
    try:
        return d.strftime("%A %-m/%-d")   # POSIX
    except ValueError:
        return d.strftime("%A %#m/%#d")   # Windows

def _select_subset(df):
    """Return a (dataframe, description) tuple based on SUBSET_STYLE."""
    if SUBSET_STYLE == "window":
        lo = pd.Timestamp(SUBSET_START)
        hi = pd.Timestamp(SUBSET_END)
        m = (df["timestamp"] >= lo) & (df["timestamp"] <= hi)
        return df.loc[m].copy(), f"time window {lo} → {hi}"
    elif SUBSET_STYLE == "around":
        c = pd.Timestamp(CENTER_TIME)
        lo = c - pd.Timedelta(minutes=RADIUS_MIN)
        hi = c + pd.Timedelta(minutes=RADIUS_MIN)
        m = (df["timestamp"] >= lo) & (df["timestamp"] <= hi)
        return df.loc[m].copy(), f"{RADIUS_MIN} min around {c}  (window {lo} → {hi})"
    else:
        raise ValueError("SUBSET_STYLE must be 'window' or 'around'")

# Sanity
if df.empty:
    print("Cleaned DataFrame `df` is empty — nothing to plot.")
else:
    # Choose data to plot
    if PLOT_SCOPE == "all":
        plot_df = df.copy()
        scope_desc = "ALL cleaned data"
    else:
        plot_df, scope_desc = _select_subset(df)
        if plot_df.empty:
            print(f"Subset selection produced 0 rows ({scope_desc}). Nothing to plot.")
            # Still show a hint of what's available:
            print("Data range:", df['timestamp'].min(), "→", df['timestamp'].max(), f"(rows={len(df)})")
        else:
            # Print and save subset rows for manual inspection
            cols_to_show = [c for c in ["timestamp","value","mode","error","source_file","vbat_mV","vin_mV","iout_mA","soc_C","rp1_C","pmic_C"] if c in plot_df.columns]
            print(f"\nSubset rows selected ({len(plot_df)} rows) — {scope_desc}")
            if len(plot_df) > PRINT_MAX_ROWS:
                print(plot_df[cols_to_show].head(PRINT_MAX_ROWS).to_string(index=False))
                print(f"... ({len(plot_df) - PRINT_MAX_ROWS} more rows not shown)")
            else:
                print(plot_df[cols_to_show].to_string(index=False))
            if SAVE_SUBSET_CSV:
                plot_df.to_csv(SUBSET_CSV_PATH, index=False)
                print(f"Saved subset to CSV: {SUBSET_CSV_PATH}")

    # If nothing to plot, stop here.
    if plot_df.empty:
        pass
    else:
        # Ensure sorted and columns present
        plot_df = plot_df.sort_values("timestamp")
        watts = plot_df[["timestamp","value"]].rename(columns={"value":"watts"}).dropna()

        # Build sunrise/sunset times
        sunrise_t = dt.datetime.strptime(SUNRISE_STR, "%H:%M:%S").time()
        sunset_t  = dt.datetime.strptime(SUNSET_STR,  "%H:%M:%S").time()

        # Figure & plot
        fig = plt.figure(figsize=FIGSIZE_IN, dpi=DPI)
        ax = fig.add_subplot(111)
        ax.plot(watts["timestamp"], watts["watts"], linewidth=0.9)
        ax.set_ylabel("Watts")
        ax.set_xlabel("Time")
        ax.set_title(f"Watts vs Time — {scope_desc}")

        # Daily major ticks, hourly minor grid
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
        ax.xaxis.set_major_formatter(FuncFormatter(_fmt_day))
        ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))
        ax.grid(which="minor", axis="x", linestyle="-", linewidth=0.5, alpha=0.3, color="0.8")

        # Night shading (outside sunrise–sunset) clipped to plot_df span
        left_data, right_data = watts["timestamp"].min(), watts["timestamp"].max()
        day_start = left_data.normalize()
        day_end   = (right_data.normalize() + pd.Timedelta(days=1))
        current = day_start
        while current < day_end:
            sr = pd.Timestamp.combine(current, sunrise_t)
            ss = pd.Timestamp.combine(current, sunset_t)
            l1, r1 = max(current, left_data), min(sr, right_data)                 # [00:00, sunrise)
            l2, r2 = max(ss, left_data), min(current + pd.Timedelta(days=1), right_data)  # [sunset, 24:00)
            if l1 < r1:
                ax.axvspan(l1, r1, alpha=0.08, zorder=0)
            if l2 < r2:
                ax.axvspan(l2, r2, alpha=0.08, zorder=0)
            current += pd.Timedelta(days=1)

        plt.tight_layout()
        plt.savefig(OUT_PNG, dpi=DPI)
        plt.close(fig)
        print(f"\nSaved {OUT_PNG}")



Saved power_watts_all_nightshade.png
